In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-187247
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-187247


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
compute_target = ComputeTarget.create(ws, "compute-cluster", compute_config)

In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,200,300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./",
            compute_target=compute_target,
            vm_size='STANDARD_D2_V2',
            entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     estimator=est,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_d5dd7fe0-8a3e-4736-bb1d-d7b0e79af47d
Web View: https://ml.azure.com/runs/HD_d5dd7fe0-8a3e-4736-bb1d-d7b0e79af47d?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-187247/workspaces/quick-starts-ws-187247&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-02-21T18:51:35.053999][API][INFO]Experiment created<END>\n""<START>[2022-02-21T18:51:35.852385][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-02-21T18:51:36.513221][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_d5dd7fe0-8a3e-4736-bb1d-d7b0e79af47d
Web View: https://ml.azure.com/runs/HD_d5dd7fe0-8a3e-4736-bb1d-d7b0e79af47d?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-187247/workspaces/quick-starts-ws-187247&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_d5dd7fe0-8a3e-4736-bb1d-d7b0e79af47d',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-21T18:51:34.718601Z',
 'endTimeUtc': '2022-02-21T19:10:13.37831Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '21971e80-ee95-4ba2-87b9-24c533b190fa',
  'user_agent': 'python/3.8.1 (Linux-5.4.0-1068-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': '44',
  'score': '0.9110773899848255',
  'best_child_run_id': 'HD_d5dd7fe0-8a3e-4736-bb1d-d7b0e79af47d_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg187247.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_d5dd7fe0-8a3e-4736-bb1d-d7b0e79af47d/azureml-logs/h

In [5]:
import joblib
# Get your best run and save the model from that run.

best_run_sdk = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run_sdk.get_metrics())

{'Regularization Strength:': 500.0, 'Max iterations:': 50, 'Accuracy': 0.9110773899848255}


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=ds,
    compute_target=compute_target,
    label_column_name="y",
    n_cross_validations=2)

In [9]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output = False)
automl_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_b016d642-70ea-4bee-aa64-e7e0fc6269c8,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_b016d642-70ea-4bee-aa64-e7e0fc6269c8',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-21T19:30:45.263645Z',
 'endTimeUtc': '2022-02-21T19:55:36.685471Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-187247","workspace_name":"quick-starts-ws-187247","region":"southcentralus","compute_target":"comput

In [14]:
# Retrieve and save your best automl model.

best_run, model = automl_run.get_output()

#print(best_run)
#print(model)

# get_metrics()
# Returns the metrics
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())

Best run metrics : {'recall_score_weighted': 0.9166009104704097, 'weighted_accuracy': 0.9546145774029606, 'f1_score_macro': 0.778437545750337, 'precision_score_weighted': 0.9124881335351966, 'AUC_weighted': 0.9453421318570883, 'balanced_accuracy': 0.7635738500522743, 'recall_score_micro': 0.9166009104704097, 'precision_score_macro': 0.7960603106496815, 'accuracy': 0.9166009104704097, 'recall_score_macro': 0.7635738500522743, 'average_precision_score_weighted': 0.9545769244605886, 'log_loss': 0.23815518412159514, 'AUC_macro': 0.9453421318570883, 'matthews_correlation': 0.5586592175911911, 'AUC_micro': 0.9801851059567424, 'norm_macro_recall': 0.5271477001045486, 'f1_score_micro': 0.9166009104704097, 'average_precision_score_micro': 0.9809340954389317, 'average_precision_score_macro': 0.8223035071481833, 'f1_score_weighted': 0.9141904952164532, 'precision_score_micro': 0.9166009104704097, 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_b016d642-70ea-4bee-aa64-e7e0fc6269c8_

In [15]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_b016d642-70ea-4bee-aa64-e7e0fc6269c8_30,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation
